## Setup

### Import Libraries

In [ ]:
%matplotlib inline

from typing import List

import warnings
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from matplotlib.ticker import ScalarFormatter

In [ ]:
warnings.filterwarnings("ignore")

matplotlib.rcParams['savefig.dpi'] = 300
matplotlib.rcParams["figure.dpi"] = 300

### Env Setup

In [ ]:
ROOT_DIR = Path()
CSV_DIR = Path().joinpath("csv_generator")
CSV_RESULTS_FILE_URL = CSV_DIR.joinpath("results.csv")

assert CSV_RESULTS_FILE_URL.exists()
assert CSV_RESULTS_FILE_URL.is_file()

## CSV Loading

In [ ]:
df = pd.read_csv(CSV_RESULTS_FILE_URL)

In [ ]:
# df.head(15)

## Utils Functions

In [ ]:
def prepare_plot_data(dfs: List[pd.DataFrame]) -> list:
    data = []

    for df in dfs:
        assert df.shape[0] > 0 and df.shape[0] <= 40

    for df in dfs:
        x, y = list(range(0, 40)), []
        for ins_index in x:
            _rows = df[df.data_index == (ins_index + 1)]
            assert _rows.shape[0] < 2
            if _rows.shape[0] == 0:
                y.append(300)
            else:
                y.append(_rows.iloc[0]["stats_tot_time"])
        data.append((x, y))

    return data

In [ ]:
def plot(title: str, subtitle: str, width: float, data: list, labels: List[str]) -> None:
    fig, ax = plt.subplots(1, 1, figsize=(20, 10))
    plt.title(title)
    fig.canvas.manager.set_window_title(subtitle)
    fig.dpi

    x_idxs = np.arange(1, 41, 1)
    for i, [_, y] in enumerate(data):
        bar = ax.bar(x_idxs+(width * i), y, width, label=labels[i], align='center')

    ax.set_xlabel("Instances")
    ax.set_ylabel("Solve Time (s)")
    ax.set_xlim(0, 41)
    ax.set_xticks(list(range(1, 41)))
    ax.set_yscale('log')
    ax.set_yticks([0.1, 1, 10, 60, 150, 300])
    ax.get_yaxis().set_major_formatter(ScalarFormatter())
    ax.grid(axis='y')
    
    ax.legend()
    plt.show()



## Plots

In [ ]:
def df_filter(
    technology, model, symmetry = False, cumulative = False, search = "linear"
) -> pd.DataFrame:
    return df.copy()[
        (df.technology == technology) & 
        (df.model == model) & 
        (df.symmetry == symmetry) & 
        (df.cumulative == cumulative) &
        (df.search == search) 
    ]

### SMT

In [ ]:
_tmp_dfs = [
    df_filter("SMT", model="base"),
    df_filter("SMT", model="base", symmetry=True),
    df_filter("SMT", model="base", cumulative=True),
]
width = 0.25 # lines width/padding
labels = [
    "base", 
    "base (symmetry)", 
    "base (cumulative)"
]

plot("SMT", "SMT", width, prepare_plot_data(_tmp_dfs), labels)

In [ ]:
_tmp_dfs = [
    df_filter("SMT", model="rotation"),
    df_filter("SMT", model="rotation", symmetry=True),
    df_filter("SMT", model="rotation", cumulative=True),
]
width = 0.25 # lines width/padding
labels = [
    "rotation", 
    "rotation (symmetry)", 
    "rotation (cumulative)",
]

plot("SMT (rotation)", "SMT (rotation)", width, prepare_plot_data(_tmp_dfs), labels)